In [ ]:
!mkdir -p /content
!curl https://raw.githubusercontent.com/urmzd/rabbit-classifier/main/resources/data.csv --output /content/data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9459  100  9459    0     0   112k      0 --:--:-- --:--:-- --:--:--  112k


In [ ]:
import pandas as pd
import requests
import os
import re
import pathlib

DATA_PATH="/content/data.csv"
IMAGE_PATH="/content/images"

table = pd.read_csv(DATA_PATH, usecols=range(2))

def init_image_directory(image_path=IMAGE_PATH) -> None:
  os.mkdir(image_path)

def download_image_to_location(link: str) -> pathlib.Path:
  file_name_regex = re.compile(r".*\/(.*\.(png|jpeg|jpg)).*")
  regex_matches = file_name_regex.match(link)

  if not regex_matches:
    raise Exception(f"Failed to match file_name for link {link}")

  file_name = regex_matches.group(1)
  image = requests.get(link)

  content_path = pathlib.Path(IMAGE_PATH)
  content_path.mkdir(parents=True, exist_ok=True)

  file_path = content_path / file_name

  with open(file_path, "wb") as handle:
    handle.write(image.content)

  return file_path


table

file_path = download_image_to_location(table.iloc[2,1])

In [ ]:
import cv2
from typing import NewType
import numpy as np

Image = NewType('Image', np.array)

def download_and_show_image(file_path: pathlib.Path) -> Image:
  img = cv2.imread(str(file_path))
  cv2_imshow(img)
  return img

download_and_show_image(file_path)

ModuleNotFoundError: No module named 'cv2'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dcd237b3-4a96-4bec-800d-0e7e0a878806' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>